In [1]:
pip uninstall mysql-connector

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip list

Package                            Version
---------------------------------- --------------------
alabaster                          0.7.12
altair                             4.2.0
anaconda-client                    1.9.0
anaconda-navigator                 2.1.4
anaconda-project                   0.10.1
anyio                              2.2.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
arrow                              0.13.1
asn1crypto                         1.4.0
astroid                            2.6.6
astropy                            4.3.1
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              21.2.0
autopep8                           1.5.7
Babel                              2.9.1
backcall                           0.2.0
backports.functools-lru-cache      1.6.4
backports.shutil-get-terminal-size 1.0.0
backports.tempfile                 

In [3]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import pandas as pd
import mysql.connector as mysql
from mysql.connector import Error

In [5]:
def DBConnect(dbName=None):
    """
    Parameters
    ----------
    dbName :
        Default value = None
    Returns
    -------
    """
    conn = mysql.connect(host='localhost', user='root', password="haira@0922787213",
                         database=dbName, buffered=True)
    cur = conn.cursor()
    return conn, cur

In [6]:
def emojiDB(dbName: str) -> None:
    conn, cur = DBConnect(dbName)
    dbQuery = f"ALTER DATABASE {dbName} CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci;"
    cur.execute(dbQuery)
    conn.commit()

In [7]:
def createDB(dbName: str) -> None:
    """
    Parameters
    ----------
    dbName :
        str
    Returns
    -------
    """
    conn, cur = DBConnect()
    cur.execute(f"CREATE DATABASE IF NOT EXISTS {dbName};")
    conn.commit()
    cur.close()

In [8]:
def createTables(dbName: str) -> None:
    """
    Parameters
    ----------
    dbName :
        str
    Returns
    -------
    """
    conn, cur = DBConnect(dbName)
    sqlFile = 'tweet_schema.sql'
    fd = open(sqlFile, 'r')
    readSqlFile = fd.read()
    fd.close()

    sqlCommands = readSqlFile.split(';')

    for command in sqlCommands:
        try:
            res = cur.execute(command)
        except Exception as ex:
            print("Command skipped: ", command)
            print(ex)
    conn.commit()
    cur.close()

    return

In [9]:
def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Parameters
    ----------
    df :
        pd.DataFrame
    Returns
    -------
    """
    cols_2_drop = ['original_text']
    try:
        df = df.drop(columns=cols_2_drop, axis=1)
        values={"hashtags": "", "user_mentions": ""}
        df = df.fillna(value=values)
        df = df.fillna(0)
    except KeyError as e:
        print("Error:", e)

    return df

In [10]:
def insert_to_tweet_table(dbName: str, df: pd.DataFrame, table_name: str) -> None:
    """
    Parameters
    ----------
    dbName :
        str
    df :
        pd.DataFrame
    table_name :
        str
    Returns
    -------
    """
    conn, cur = DBConnect(dbName)

    df = preprocess_df(df)

    for _, row in df.iterrows():
        sqlQuery = f"""INSERT INTO {table_name} (created_at, source, clean_text, sentiment, polarity,  language,
                    favorite_count, retweet_count, original_author, followers_count, friends_count, possibly_sensitive, hashtags, user_mentions, place)
             VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s );"""
        data = (row[0], row[1], row[2], row[3], (row[4]), (row[5]), row[6], row[7], row[8], row[9], row[10],
                row[11], row[12], row[13], row[14])

        try:
            # Execute the SQL command
            cur.execute(sqlQuery, data)
            # Commit your changes in the database
            conn.commit()
            print("Data Inserted Successfully")
        except Exception as e:
            conn.rollback()
            print("Error: ", e)
    return

In [11]:
def db_execute_fetch(*args, many=False, tablename='', rdf=True, **kwargs) -> pd.DataFrame:
    """
    Parameters
    ----------
    *args :
    many :
         (Default value = False)
    tablename :
         (Default value = '')
    rdf :
         (Default value = True)
    **kwargs :
    Returns
    -------
    """
    connection, cursor1 = DBConnect(**kwargs)
    if many:
        cursor1.executemany(*args)
    else:
        cursor1.execute(*args)

    # get column names
    field_names = [i[0] for i in cursor1.description]

    # get column values
    res = cursor1.fetchall()

    # get row count and show info
    nrow = cursor1.rowcount
    if tablename:
        print(f"{nrow} records fetched from {tablename} table")

    cursor1.close()
    connection.close()

    # return result
    if rdf:
        return pd.DataFrame(res, columns=field_names)
    else:
        return res

In [12]:
if __name__ == "__main__":
    createDB(dbName='tweets')
    emojiDB(dbName='tweets')
    createTables(dbName='tweets')

    df = pd.read_csv('../clean_tweet_data.csv')
    insert_to_tweet_table(dbName='tweets', df=df, table_name='TweetInformation')


ProgrammingError: 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)